# **The Card class**

This notebook provides some documentation about ```Card``` objects.

Run the following codeblocks to import the ```Card``` class into this notebook.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"): 
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: d:\My Projects\Programming\Game-Development\fivecarddraw


In [2]:
from fivecarddraw import Card

## **Card Representation**

```Card``` objects have the following constants for representation:

* ```Card.VALUES = ("2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A")```
* ```Card.SUITS = ("♠", "♡", "♢", "♣")```

When a ```Card``` is initialised, it takes two arguments; namely, ```value``` and  ```suit```. These should be integers, which can be used as indices to retrieve a representation for the card. Those indices are stored as attributes; namely, ```Card.value_i``` and ```Card.suit_i``` respectfully. Then the representation attributes can be created as follows:

* ```Card.value_r = Card.VALUES[Card.value_i]```
* ```Card.suit_r = Card.SUITS[Card.suit_i]```
* ```Card.r = Card.value_r + Card.suit_r```
 

If a ```Card``` object is printed, it's represented as the ```string``` stored as ```Card.r```. Furthermore ```str(Card) == Card.r```.

### **Example - Creating an Ace of Spades**

In [4]:
ace_of_spades = Card(12,0)
print(f"Value pool: {ace_of_spades.VALUES}")
print(f"Suit pool: {ace_of_spades.SUITS}")
print(f"value_i: {ace_of_spades.value_i}, suit_i: {ace_of_spades.suit_i}")
print(f"value_r: {ace_of_spades.value_r}, suit_r: {ace_of_spades.suit_r}")
print(f"Card: {ace_of_spades}")

Value pool: ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
Suit pool: ('♠', '♡', '♢', '♣')
value_i: 12, suit_i: 0
value_r: A, suit_r: ♠
Card: A♠


### **Example - Creating a Random Card**

In [5]:
from random import randint

def RandomCard():
    value = randint(0,12)
    suit = randint(0,3)
    return Card(value, suit)

In [6]:
card = RandomCard()
print(f"Value pool: {card.VALUES}")
print(f"Suit pool: {card.SUITS}")
print(f"value_i: {card.value_i}, suit_i: {card.suit_i}")
print(f"value_r: {card.value_r}, suit_r: {card.suit_r}")
print(f"Random Card: {card}")

Value pool: ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
Suit pool: ('♠', '♡', '♢', '♣')
value_i: 8, suit_i: 1
value_r: 10, suit_r: ♡
Random Card: 10♡


## **Card Optimisation**

In order to boost the time-efficieny of hand-ranking algorithms, it's extremely beneficial to encode certain details about cards into a 32-bit integer, from which card details can then be efficiently retrieved. For more information about the hand-ranking algorithm itself, see: [handtracker.ipynb](handtracker.ipynb). For more information about the encoding of card details as a 32-bit integer, continue below.

```Card``` objects have the following constants for optimisation:

* ```Card.PRIMES = (2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41)```

The semantic mask used to encode each card is stored in the following constant:

*  ```Card.MASK = "xxxAKQJT98765432♣♢♡♠RRRRxxPPPPPP"```

The ```Card.MASK``` attribute is split into four main parts:

* ```AKQJT98765432``` where the value of the card is stored as a 13-bit integer.
* ```♣♢♡♠``` where the suit of the card is stored as a 4-bit integer.
* ```RRRR``` where the value of the card is stored as a 4-bit integer.
* ```PPPPP``` where the value of the card is stored as a 6-bit prime number.



Each section is computed upon the initialisation of the ```Card``` object as follows:

* ```Card._value = (2 ** Card.value_i) << 16```
* ```Card._suit = (2 ** Card.suit_i) << 12```
* ```Card._rank = Card.value_i << 8```
* ```Card._prime = Card.PRIMES[Card.value_i]```

Then the full encoding of the card is computed and stored into the following attribute:

* ```Card.b = Card._value + Card._suit + Card._rank + Card._prime```

Furthermore ```int(Card) == Card.b```.

### **Example - Encoding an Ace of Spades**

In [3]:
ace_of_spades = Card(12,0)
print(f"Values pool: {ace_of_spades.VALUES}")
print(f"Primes pool: {ace_of_spades.PRIMES}")
print(f"AKQJT98765432: {bin(ace_of_spades._value >> 16)[2:].zfill(13)}")
print(f"♣♢♡♠: {bin(ace_of_spades._suit >> 12)[2:].zfill(4)}")
print(f"RRRR: {bin(ace_of_spades._rank >> 8)[2:].zfill(4)} (as a decimal number: {ace_of_spades._rank >> 8})")
print(f"PPPPPP: {bin(ace_of_spades._prime)[2:].zfill(6)} (as a decimal number: {ace_of_spades._prime})")
print(f"Card: {ace_of_spades}")
print(f"32-bit encoding: {bin(ace_of_spades.b)[2:].zfill(32)}")
print(f"integer representation: {int(ace_of_spades)}")

Values pool: ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
Primes pool: (2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41)
AKQJT98765432: 1000000000000
♣♢♡♠: 0001
RRRR: 1100 (as a decimal number: 12)
PPPPPP: 101001 (as a decimal number: 41)
Card: A♠
32-bit encoding: 00010000000000000001110000101001
integer representation: 268442665


### **Example - Encoding a Random Card**

In [5]:
from random import randint

def RandomCard():
    value = randint(0,12)
    suit = randint(0,3)
    return Card(value, suit)

In [9]:
card = RandomCard()
print(f"Values pool: {card.VALUES}")
print(f"Primes pool: {card.PRIMES}")
print(f"AKQJT98765432: {bin(card._value >> 16)[2:].zfill(13)}")
print(f"♣♢♡♠: {bin(card._suit >> 12)[2:].zfill(4)}")
print(f"RRRR: {bin(card._rank >> 8)[2:].zfill(4)} (as a decimal number: {card._rank >> 8})")
print(f"PPPPPP: {bin(card._prime)[2:].zfill(6)} (as a decimal number: {card._prime})")
print(f"Card: {card}")
print(f"32-bit encoding: {bin(card.b)[2:].zfill(32)}")
print(f"integer representation: {int(card)}")

Values pool: ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
Primes pool: (2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41)
AKQJT98765432: 0001000000000
♣♢♡♠: 0001
RRRR: 1001 (as a decimal number: 9)
PPPPPP: 011101 (as a decimal number: 29)
Card: J♠
32-bit encoding: 00000010000000000001100100011101
integer representation: 33560861


### **Example - Encoding a Selected Card**

In [14]:
value = int(input("Choose a value for the card from 0 to 12"))
suit = int(input("Choose a value for the card from 0 to 3"))
card = Card(value, suit)

In [8]:
print(f"Values pool: {card.VALUES}")
print(f"Primes pool: {card.PRIMES}")
print(f"AKQJT98765432: {bin(card._value >> 16)[2:].zfill(13)}")
print(f"♣♢♡♠: {bin(card._suit >> 12)[2:].zfill(4)}")
print(f"RRRR: {bin(card._rank >> 8)[2:].zfill(4)} (as a decimal number: {card._rank >> 8})")
print(f"PPPPPP: {bin(card._prime)[2:].zfill(6)} (as a decimal number: {card._prime})")
print(f"Card: {card}")
print(f"32-bit encoding: {bin(card.b)[2:].zfill(32)}")
print(f"integer representation: {int(card)}")

Values pool: ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
Primes pool: (2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41)
AKQJT98765432: 0000010000000
♣♢♡♠: 0001
RRRR: 0111 (as a decimal number: 7)
PPPPPP: 010011 (as a decimal number: 19)
Card: 9♠
32-bit encoding: 00000000100000000001011100010011
integer representation: 8394515
